# ETL da camada silver para camada gold

Esta célula importa todas as bibliotecas necessárias para o processo de ETL.
Aqui são carregados os pacotes para manipulação de dados (pyspark), conexão com o banco de dados PostgreSQL (psycopg), controle de mensagens de erro (sys) e tratamento de avisos (warnings).
Ela deve ser executada antes de qualquer outra célula, pois fornece as dependências básicas que serão usadas nas etapas de Extract, Transform e Load.

In [1]:
import os
import sys
import warnings
import psycopg2 
import pandas as pd
import numpy as np
from psycopg2 import sql
from dotenv import load_dotenv 

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, IntegerType, DoubleType, DecimalType, TimestampType

# 1. CONFIGURAÇÕES GERAIS

como costume e para evitar possivei erro preferismo configurar o todo sparky logo no topo para ter tudo que precisamos pronto 
aproveitamos e nessa parte para Ignora avisos chatos de versão/depreciação e tambem arrega variáveis da .env


In [2]:

warnings.filterwarnings('ignore') 
load_dotenv() 

False

### 1.1 DRIVERS E CAMINHOS
nessa parte e onde feito o ajuste o caminho se necessário. O driver JDBC é vital para o Spark falar com o Postgres.

In [3]:

JAR_NAME = "postgresql-42.7.8.jar"
JAR_PATH = os.path.abspath(JAR_NAME)

if not os.path.exists(JAR_PATH):
    print(f"⚠️ AVISO: Driver JDBC '{JAR_NAME}' não encontrado na pasta atual.")
    print("O Spark não conseguirá salvar no banco sem ele.")
else:
    print(f"✅ Driver JDBC localizado: {JAR_PATH}")

✅ Driver JDBC localizado: /home/emivalto/workspace/BD2V/SDBD2---INEP/Transformer/postgresql-42.7.8.jar


### 1.2 INICIALIZAÇÃO DA SESSÃO SPARK
 esta e uma das fasses cruciais tambem pois e onde finalizamos sessão antiga se existir para garantir que o JAR seja carregado corretamente, e onde configuramos o Spark

In [4]:


try:
    spark.stop()
except:
    pass

print("\n Iniciando motor Spark...")

spark = SparkSession.builder \
    .appName("ETL_Enem_Gold_Layer") \
    .config("spark.driver.memory", "5g") \
    .config("spark.jars", JAR_PATH) \
    .config("spark.driver.extraClassPath", JAR_PATH) \
    .config("spark.executor.extraClassPath", JAR_PATH) \
    .config("spark.sql.legacy.timeParserPolicy", "CORRECTED") \
    .getOrCreate()

print(f"Spark {spark.version} ativo e pronto para análise!")



 Iniciando motor Spark...


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/22 21:01:08 WARN Utils: Your hostname, Emivalto, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/22 21:01:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
26/01/22 21:01:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark 4.1.1 ativo e pronto para análise!


### 1.3 VARIÁVEIS DE CONEXÃO


In [5]:

DB_CONFIG = {
    "host": "localhost",
    "port": "5432",
    "dbname": "dados_inep",
    "user": "admin",
    "password": "l1l2r1r2" 
}

JDBC_URL = f"jdbc:postgresql://{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"

# 1. Extract

Esta célula define as configurações de conexão com o banco de dados PostgreSQL e monta a consulta SQL que será usada para extrair os dados.
Ela cria variáveis com credenciais, monta o nome completo da tabela (schema.tabela) e gera a query SELECT * FROM silver.listings, além de preparar a connection string usada na etapa de conexão.

In [ ]:

DB_SCHEMA = "silver"
TABLE_NAME = "microdados_enem"
FULL_TABLE_NAME = f"{DB_SCHEMA}.{TABLE_NAME}" 


def get_jdbc_connection_info():
    load_dotenv()
    
    url = os.getenv('DB_JDBC_URL') 
    if url:
        return url

    DB_USER = "admin"
    DB_PASS = "l1l2r1r2"
    DB_HOST = "localhost"
    DB_PORT = "5432"
    DB_NAME = "dados_inep"

    return f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}", DB_USER, DB_PASS

jdbc_url, db_user, db_pass = get_jdbc_connection_info()

print(f" Preparando leitura da tabela: {FULL_TABLE_NAME}")
print(f" URL de Conexão: {jdbc_url}")

try:
    df_silver = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", FULL_TABLE_NAME) \
        .option("user", db_user) \
        .option("password", db_pass) \
        .option("driver", "org.postgresql.Driver") \
        .load()

    print("\n Sucesso! Dados da camada Silver carregados.")
    print("Schema carregado do Banco:")
    df_silver.printSchema()

except Exception as e:
    print(f" Erro ao ler do banco: {e}")

📡 Preparando leitura da tabela: silver.microdados_enem
🔗 URL de Conexão: jdbc:postgresql://localhost:5432/dados_inep

 Sucesso! Dados da camada Silver carregados.
Schema carregado do Banco:
root
 |-- nu_inscricao: long (nullable = true)
 |-- tp_faixa_etaria: integer (nullable = true)
 |-- tp_sexo: string (nullable = true)
 |-- tp_estado_civil: integer (nullable = true)
 |-- tp_cor_raca: integer (nullable = true)
 |-- tp_nacionalidade: integer (nullable = true)
 |-- tp_st_conclusao: integer (nullable = true)
 |-- tp_ano_concluiu: integer (nullable = true)
 |-- tp_escola: integer (nullable = true)
 |-- in_treineiro: integer (nullable = true)
 |-- co_municipio_prova: integer (nullable = true)
 |-- no_municipio_prova: string (nullable = true)
 |-- co_uf_prova: integer (nullable = true)
 |-- sg_uf_prova: string (nullable = true)
 |-- tp_presenca_cn: integer (nullable = true)
 |-- tp_presenca_ch: integer (nullable = true)
 |-- tp_presenca_lc: integer (nullable = true)
 |-- tp_presenca_mt: in

# 2 CARGA DOS DADOS (Leitura JDBC via Spark) 

Esta proxima célula ira executa a extração dos dados do banco PostgreSQL.

Ela tem que estabelece a conexão usando as configurações definidas anteriormente, converte o objeto SQL em uma query, executa a consulta e carrega o resultado no DataFrame df.

Em caso de falha na conexão ou na leitura, deve ser exibido uma mensagem de erro detalhada e encerra o processo.

Em Big Data, constumasse usar o `spark.read` para criar um ponteiro distribuído para os dados, mais robusto.

afim de melhorar e ver o se aconteceu algum erro no final e feito umas captura dos erros genéricos do Java/Spark.

In [7]:


try:
    print(" Estabelecendo conexão com o Spark JDBC...")
    
    df = spark.read \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", FULL_TABLE_NAME) \
        .option("user", db_user) \
        .option("password", db_pass) \
        .option("driver", "org.postgresql.Driver") \
        .load()

    print(" Conexão estabelecida.")
    print(f" Tabela '{FULL_TABLE_NAME}' mapeada com sucesso para o Spark!")
    

except Exception as e:
   
    print(f"\n Ocorreu um erro ao conectar ou ler o banco de dados via Spark ")
    print(f"Erro detalhado: {e}")
    sys.exit(1)

 Estabelecendo conexão com o Spark JDBC...
 Conexão estabelecida.
 Tabela 'silver.microdados_enem' mapeada com sucesso para o Spark!


### 2.1 VALIDAÇÃO DA CARGA
Estas células e basicamente para validar a quantidade de dados esta vindo corretamente e exibem um resumo simples do resultado da extração, mostrando o número total de registros carregados no DataFrame df, as primeiras cinco tuplas e os tipos de cada dado.
Elas servem para confirmar visualmente que a consulta foi executada com sucesso e quantas linhas foram retornadas do banco.

In [10]:

print(" Contando registros no DataFrame...")
qtd_linhas = df.count()
print(f"Total de linhas carregadas: {qtd_linhas}")

print("\n --- Estrutura da Tabela (Schema) ---")

df.printSchema()

print("\n--- Visualização dos Dados (Amostra) ---")

df.show(5, truncate=False)



 Contando registros no DataFrame...
Total de linhas carregadas: 509954

 --- Estrutura da Tabela (Schema) ---
root
 |-- nu_inscricao: long (nullable = true)
 |-- tp_faixa_etaria: integer (nullable = true)
 |-- tp_sexo: string (nullable = true)
 |-- tp_estado_civil: integer (nullable = true)
 |-- tp_cor_raca: integer (nullable = true)
 |-- tp_nacionalidade: integer (nullable = true)
 |-- tp_st_conclusao: integer (nullable = true)
 |-- tp_ano_concluiu: integer (nullable = true)
 |-- tp_escola: integer (nullable = true)
 |-- in_treineiro: integer (nullable = true)
 |-- co_municipio_prova: integer (nullable = true)
 |-- no_municipio_prova: string (nullable = true)
 |-- co_uf_prova: integer (nullable = true)
 |-- sg_uf_prova: string (nullable = true)
 |-- tp_presenca_cn: integer (nullable = true)
 |-- tp_presenca_ch: integer (nullable = true)
 |-- tp_presenca_lc: integer (nullable = true)
 |-- tp_presenca_mt: integer (nullable = true)
 |-- co_prova_cn: integer (nullable = true)
 |-- co_prov

+------------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------------------------------------+---------------------------------------------+--------------------------------------------------+---------------------------------------------+---------+---------------------------------------------+---------------------------------------------+--------------------------------------------------+---------------------------------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|nu_inscricao|tp_faixa_etaria|

# 2. Transform
